**Trabalho em Grupo**

Link Edmodo: https://new.edmodo.com/post/696551019

**Problema**

> Um sistema de classificação de tópicos consiste em um modelo capaz de identificar a qual domínio uma determinada sentença ou texto pertence. Crie um classificador que receba uma frase em inglês e indique se ela faz parte de uma das seguintes categorias: aventura, ficção científica, religião ou governo, utilizando o corpus em anexo.

> Use as etapas de pré-processamento que vimos em sala e representações de texto diferentes se desejar (bag-of-words/LSA, Skip-gram, CBOW, GloVe). Utilize os algoritmos de classificação que desejar, e faça seleção de hiperparâmetros. Siga sempre as boas práticas para experimentos de aprendizado de máquina para evitar underfitting e overfitting.

> O grupo que obtiver o melhor resultado no corpus de teste (de posse somente do professor) receberá 1 ponto a mais na média de disciplina. Se mais de um grupo tiver o mesmo melhor resultado, nenhum grupo ganhará essa pontuação.



Nome dos integrantes:

* Aluno 1: Aline Cristini - 183132
* Aluno 2: Camila Rodrigues - 183143
* Aluno 3: Rafael Gimenes Leite - 101634




In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['corpus_categorias_treino.csv']


In [2]:
cp_cat_train = pd.read_csv('../input/corpus_categorias_treino.csv')
cp_cat_train.rename(columns={'Unnamed: 0': 'idx'}, inplace=True)

In [3]:
#listando categorias y 
cp_cat_train['category'].unique()

array(['adventure', 'science_fiction', 'religion', 'government'],
      dtype=object)

In [4]:
cp_cat_train

,idx,words,category
0,0,He certainly didn't want a wife who was fickle...,adventure
1,1,"If he had married her , he'd have been asking ...",adventure
2,2,Sometimes he woke up in the middle of the nigh...,adventure
3,3,His plans and dreams had revolved around her s...,adventure
4,4,The best antidote for the bitterness and disap...,adventure
5,5,He found that if he was tired enough at night ...,adventure
6,6,Each day he found himself thinking less often ...,adventure
7,7,"each day the hurt was a little duller , a litt...",adventure
8,8,He had plenty of work to do .,adventure
9,9,"The grass in the meadows came fast , now that ...",adventure


In [5]:
#y_data = []
#x_data = []
#separando x e Y binarizando
#adventure=0,science_fiction=1,religion=2,government=3
#for i,r in cp_cat_train.iterrows():
#    sentencas = r['words']
#    cat = r['category']   
#    if (cat=='adventure'):
#        y=0
#    elif (cat=='science_fiction'):
#        y=1
#    elif (cat=='religion'):
#        y=2
#    elif (cat=='government'):
#        y=3
#    y_data.append(y)
#    x_data.append(sentencas)


<b>TF-IDF (Term Frequency - Inverse Document Frequency)</b>

In [6]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0].lower(), pos))

    return lemmas

In [7]:
tokenizado = []
for i in cp_cat_train['words']:
    tokenizado.append(my_tokenizer(i))
    

In [8]:
tokenizado

[['certainly', "n't", 'want', 'wife', 'fickle', 'ann'],
 ['marry', "'d", 'ask', 'trouble'],
 ['sometimes',
  'wake',
  'middle',
  'night',
  'thinking',
  'ann',
  'could',
  'get',
  'back',
  'sleep'],
 ['plan', 'dream', 'revolve', 'around', 'much', 'long', 'felt', 'nothing'],
 ['best',
  'antidote',
  'bitterness',
  'disappointment',
  'poison',
  'hard',
  'work'],
 ['find',
  'tire',
  'enough',
  'night',
  'go',
  'sleep',
  'simply',
  'exhaust',
  'stay',
  'awake'],
 ['day', 'find', 'think', 'less', 'often', 'ann'],
 ['day', 'hurt', 'little', 'duller', 'little', 'less', 'poignant'],
 ['plenty', 'work'],
 ['grass', 'meadow', 'come', 'fast', 'warm', 'weather'],
 ['could',
  'afford',
  'lose',
  'drop',
  'precious',
  'water',
  'spend',
  'waking',
  'hour',
  'along',
  'ditch',
  'meadow'],
 ['idea', 'much', 'time', 'budd', 'would', 'give'],
 ['stop',
  'every',
  'minute',
  'lean',
  'shovel',
  'study',
  'horizon',
  'nothing',
  'happen',
  'day',
  'drag',
  'monoto

In [9]:
frase = list()
for i in tokenizado:
    frase.append(' '.join(i))
    
cp_cat_train['words_2'] = frase
cp_cat_train.category.replace(['adventure', 'government','religion','science_fiction'], [0,1,2,3], inplace=True)


In [10]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)

tfs = tfidf_vectorizer.fit_transform(cp_cat_train["words_2"])

print(tfs.shape)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
(8200, 10779)


In [11]:
print([k for k in tfidf_vectorizer.vocabulary_.keys()][:20])

['certainly', "n't", 'want', 'wife', 'fickle', 'ann', 'marry', "'d", 'ask', 'trouble', 'sometimes', 'wake', 'middle', 'night', 'think', 'could', 'get', 'back', 'sleep', 'plan']


In [12]:
print(tfs[:50,:50])

  (1, 2)	0.4465555567414524
  (28, 8)	0.38369819378505776
  (28, 0)	0.3158108868475342
  (37, 8)	0.11556834142587807
  (38, 0)	0.15708306505836145
  (43, 0)	0.37764526914596375
  (44, 8)	0.29157743621009474
  (45, 0)	0.1994899491813105
  (46, 0)	0.704519768403376
  (47, 0)	0.308933536886169


**TF-IDF de N-gramas**

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(ngram_range=(3,3))

n_gram_counts = count_vect.fit_transform(cp_cat_train["words_2"])

tfidf_transformer = TfidfTransformer()

tfs_ngrams = tfidf_transformer.fit_transform(n_gram_counts)

print(tfs_ngrams.shape)

(8200, 52897)


<b>Redução de Dimensionalidade</b>

In [14]:
# Dividindo os dados
from sklearn.model_selection import train_test_split
x_train, x_test , y_train, y_test = train_test_split(cp_cat_train['words'], cp_cat_train["category"], random_state = 42)

In [15]:
from sklearn.decomposition import TruncatedSVD

class SVDDimSelect(object):
    def fit(self, X, y=None):        
        try:
            self.svd_transformer = TruncatedSVD(n_components=round(X.shape[1]/2))
            self.svd_transformer.fit(X)
        
            cummulative_variance = 0.0
            k = 0
            for var in sorted(self.svd_transformer.explained_variance_ratio_)[::-1]:
                cummulative_variance += var
                if cummulative_variance >= 0.5:
                    break
                else:
                    k += 1
                
            self.svd_transformer = TruncatedSVD(n_components=k)
        except Exception as ex:
            print(ex)
            
        return self.svd_transformer.fit(X)
    
    def transform(self, X, Y=None):
        return self.svd_transformer.transform(X)
        
    def get_params(self, deep=True):
        return {}

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=10, weights='uniform')

my_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=my_tokenizer)),\
                       ('svd', SVDDimSelect()), \
                       ('clf', clf)])

In [17]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

par = {'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']}

hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='accuracy', n_jobs=1, n_iter=20)

In [18]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

par = {'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']}


hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='accuracy', n_jobs=1, n_iter=20)

In [19]:
hyperpar_selector.fit(X=x_train, y=y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...i',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
          fit_params=None, iid='warn', n_iter=20, n_jobs=1,
          param_distributions={'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [20]:
print("Best score: %0.3f" % hyperpar_selector.best_score_)
print("Best parameters set:")
best_parameters = hyperpar_selector.best_estimator_.get_params()
for param_name in sorted(par.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.559
Best parameters set:
	clf__n_neighbors: 7
	clf__weights: 'distance'


**Testando o classificador**

In [21]:
from sklearn.metrics import *

y_pred = hyperpar_selector.predict(x_test)

print(accuracy_score(y_test, y_pred))

0.5785365853658536


**Serializando o modelo**

In [22]:
import pickle

string_obj = pickle.dumps(hyperpar_selector)

In [23]:
model_file = open('model.pkl', 'wb')

model_file.write(string_obj)

model_file.close()

**Abrindo e usando um modelo salvo **

In [24]:
model_file = open('model.pkl', 'rb')
model_content = model_file.read()

obj_classifier = pickle.loads(model_content)

model_file.close()

res = obj_classifier.predict(["what's up bro?"])

print(res)

[0]


In [25]:
res = obj_classifier.predict(x_test)
print(accuracy_score(y_test, res))

0.5785365853658536


**SKIP GRAM**

In [26]:
from gensim.models import Word2Vec

#sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.


class Word2VecTransformer(object):
    
    ALGO_SKIP_GRAM=1
    ALGO_CBOW=2    
    
    def __init__(self, algo=1):
        self.algo = algo
    
    def fit(self, X, y=None):     
        X = [nltk.word_tokenize(x) for x in X]
        
        self.word2vec = Word2Vec(X, min_count=2, sg=1)
        
        # Pegamos a dimensão da primeira palavra, para saber quantas dimensões estamos trabalhando,
        # assim podemos ajustar nos casos em que aparecerem palavras que não existirem no vocabulário.
        first_word = next(iter(self.word2vec.wv.vocab.keys()))
        self.num_dim = len(self.word2vec[first_word])       
        
        return self
    
    def transform(self, X, Y=None):        
        X = [nltk.word_tokenize(x) for x in X]
        
        return np.array([np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.num_dim)], axis=0) 
                         for words in X])

        
    def get_params(self, deep=True):
        return {}

In [27]:
from sklearn.pipeline import Pipeline
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=10, weights='uniform')

my_pipeline = Pipeline([('w2vt', Word2VecTransformer()), \
                       ('clf', clf)])

In [28]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

par = {'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']}


hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='accuracy', n_jobs=1, n_iter=20)

In [29]:
hyperpar_selector.fit(X=x_train, y=y_train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launche

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('w2vt', <__main__.Word2VecTransformer object at 0x7fb4e6cc9898>), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
          fit_params=None, iid='warn', n_iter=20, n_jobs=1,
          param_distributions={'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [30]:
print("Best score: %0.3f" % hyperpar_selector.best_score_)
print("Best parameters set:")
best_parameters = hyperpar_selector.best_estimator_.get_params()
for param_name in sorted(par.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.694
Best parameters set:
	clf__n_neighbors: 28
	clf__weights: 'distance'


In [31]:
from sklearn.metrics import *

y_pred = hyperpar_selector.predict(x_test)

print(accuracy_score(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


0.7185365853658536


**Arquitetura CBOW (Continuous Bag of Words)**

In [32]:
from gensim.models import Word2Vec

#sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.


class Word2VecTransformer(object):
    
    ALGO_SKIP_GRAM=1
    ALGO_CBOW=2    
    
    def __init__(self, algo=2):
        self.algo = algo
    
    def fit(self, X, y=None):     
        X = [nltk.word_tokenize(x) for x in X]
        
        self.word2vec = Word2Vec(X, min_count=2, sg=1)
        
        # Pegamos a dimensão da primeira palavra, para saber quantas dimensões estamos trabalhando,
        # assim podemos ajustar nos casos em que aparecerem palavras que não existirem no vocabulário.
        first_word = next(iter(self.word2vec.wv.vocab.keys()))
        self.num_dim = len(self.word2vec[first_word])       
        
        return self
    
    def transform(self, X, Y=None):        
        X = [nltk.word_tokenize(x) for x in X]
        
        return np.array([np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.num_dim)], axis=0) 
                         for words in X])

        
    def get_params(self, deep=True):
        return {}

In [33]:
from sklearn.pipeline import Pipeline
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=10, weights='uniform')

my_pipeline = Pipeline([('w2vt', Word2VecTransformer()), \
                       ('clf', clf)])

In [34]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

par = {'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']}


hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='accuracy', n_jobs=1, n_iter=20)

In [35]:
hyperpar_selector.fit(X=x_train, y=y_train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launche

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('w2vt', <__main__.Word2VecTransformer object at 0x7fb4d866ffd0>), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform'))]),
          fit_params=None, iid='warn', n_iter=20, n_jobs=1,
          param_distributions={'clf__n_neighbors': range(1, 60), 'clf__weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [36]:
print("Best score: %0.3f" % hyperpar_selector.best_score_)
print("Best parameters set:")
best_parameters = hyperpar_selector.best_estimator_.get_params()
for param_name in sorted(par.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.691
Best parameters set:
	clf__n_neighbors: 21
	clf__weights: 'distance'


In [37]:
from sklearn.metrics import *

y_pred = hyperpar_selector.predict(x_test)

print(accuracy_score(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


0.7248780487804878


A maior ACURÁCIA obtida foi de **0.7219512195121951** com Arquitetura CBOW.